In [ ]:
# Generic imports
import os, sys, glob, tempfile, pickle
import numpy as np
import scipy.linalg as sl, scipy.optimize as so
import matplotlib.pyplot as plt
import numdifftools as nd
import corner

# Non-traditional packages
import libstempo as lt

# The actual outlier code
import interval as itvl
from nutstrajectory import nuts6

# Setting up the Likelihood

First pick your pulsar and locate the corresponding par and tim files. By default this cell will look in your current working directoy, but it can always be redefined as necessary

In [ ]:
psr = 'B1855+09'
parfile = glob.glob(f'./{psr}*.par')[0]
timfile = glob.glob(f'./{psr}*.tim')[0]

This creates an Interval Likelihood object, which will load/process the pulsar data and perform a few coordinate transformations to get it ready for HMC. If you're curious about the output, just uncomment the second line.

In [ ]:
likob = itvl.Interval(parfile, timfile)
# print(likob.full_loglikelihood_grad(likob.pstart))

The next step is to get an approximate maximum of the posterior. First we'll split our log_likelihood_grad function into two parts: one that returns the ll, and one that returns the gradient.

In [ ]:
def func(x):
    ll, _ = likob.full_loglikelihood_grad(x)
    
    return -np.inf if np.isnan(ll) else ll

def jac(x):
    _, j = likob.full_loglikelihood_grad(x)
    return j

What's the log likelihood for our starting parameter vector?

In [ ]:
func(likob.pstart)

Now we compute the approximate maximum, and save it to a pickle file if it doesn't already exist

In [ ]:
endpfile = psr + '-endp.pickle'

In [ ]:
%%time
if not os.path.isfile(endpfile):
    endp = likob.pstart
    for iter in range(3):
        res = so.minimize(lambda x: -func(x),
                          endp,
                          jac=lambda x: -jac(x),
                          hess=None,
                          method='L-BFGS-B', options={'disp': True})

        endp = res['x']
    pickle.dump(endp,open(endpfile,'wb'))
else:
    endp = pickle.load(open(endpfile,'rb'))

Our approximate maximum should be larger. You can double-check that here

In [ ]:
func(endp)

Next we want to 'whiten' our likelihood (a reparameterization where our new covariance matrix is the identity matrix). To that we'll need to compute the Hessian of our posterior. This calculation may take some time relative to the maximization function above.

In [ ]:
nhyperpars = likob.ptadict[likob.pname + '_outlierprob'] + 1

In [ ]:
hessfile = psr + '-fullhessian.pickle'

In [ ]:
%%time
if not os.path.isfile(hessfile):
    reslice = np.arange(0,nhyperpars)

    def partfunc(x):
        p = np.copy(endp)
        p[reslice] = x
        return likob.full_loglikelihood_grad(p)[0]

    ndhessdiag = nd.Hessdiag(func)
    ndparthess = nd.Hessian(partfunc)

    # Create a good-enough approximation for the Hessian
    nhdiag = ndhessdiag(endp)
    nhpart = ndparthess(endp[reslice])
    fullhessian = np.diag(nhdiag)
    fullhessian[:nhyperpars,:nhyperpars] = nhpart
    pickle.dump(fullhessian,open(hessfile,'wb'))
else:
    fullhessian = pickle.load(open(hessfile,'rb'))

With the Hessian in hand, we can whiten our likelihood object.

In [ ]:
wl = itvl.whitenedLikelihood(likob, endp, -fullhessian)

This new likelihood object should have the same value as the old Interval one. As a sanity check, you can run this below to check.

In [ ]:
likob.pstart = endp
wlps = wl.forward(endp)
print(likob.full_loglikelihood_grad(endp))
print(wl.likob.full_loglikelihood_grad(wl.backward(wlps)))

# NUTS Sampler

Now it's time to sample. Set up the directory where you want the chains to be stored (or let this code make it for you), and define the number of samples your want to run for. The default is 20,000. You can also set the length of the burn-in, which is defaulted to 1,000 samples.

In [ ]:
chaindir = 'chains_' + psr
Nsamples = 20000
Nburnin = 1000

In [ ]:
!mkdir -p {chaindir}

In [ ]:
%%time
chainfile = chaindir + '/samples.txt'
if not os.path.isfile(chainfile) or len(open(chainfile,'r').readlines()) < 19999:
    # Run NUTS for 20000 samples, with a burn-in of 1000 samples (target acceptance = 0.6)
    samples, lnprob, epsilon = nuts6(wl.loglikelihood_grad, Nsamples, Nburnin,
                                     wlps, 0.6,
                                     verbose=True,
                                     outFile=chainfile,
                                     pickleFile=chaindir + '/save')

# Post-Processing

With the sampling complete, we can now analyze the chains and look for outliers in our data. The first step is undo all of our coordinate transformations to get back to real, tangible values of our parameters. Each NUTS sample generated gives one parameter vector, and the full array for all samples is saved to a .npy object

In [ ]:
parsfile = psr + '-pars.npy'

In [ ]:
%%time
if not os.path.isfile(parsfile):
    samples = np.loadtxt(chaindir + '/samples.txt')
    fullsamp = wl.backward(samples[:,:-2])
    funnelsamp = likob.backward(fullsamp)
    pars = likob.multi_full_backward(funnelsamp)
    np.save(parsfile,pars)
else:
    pars = np.load(parsfile)

First let's look at a corner plot of the posteriors of all our hyperparameters (including the new outlier parameter).

In [ ]:
parnames = list(likob.ptadict.keys())
if not os.path.isfile(psr + '-corner.pdf'):
    corner.corner(pars[:,:nhyperpars],labels=parnames[:nhyperpars],show_titles=True);
    plt.savefig(psr + '-corner.pdf')

Now we finally look for outliers. The NUTS sampler used an outlier-robust likelihood, so our job now is to compute some 'outlier probability' for each observation in the dataset. We will get a vector of these outlier probabilities, one per TOA, and a vector of the corresponding uncertainties in this outlier probability. This calculation uses the pulse period of the pulsar.

In [ ]:
likob.P0

In [ ]:
def poutlier(p,likob):
    """Invoked on a sample parameter set and the appropriate likelihood,
    returns the outlier probability (a vector over the TOAs) and
    the individual sqrt(chisq) values"""
    
    # invoke the likelihood
    _, _ = likob.base_loglikelihood_grad(p)

    # get the piccard pulsar object
    # psr = likob.psr

    r = likob.detresiduals
    N = likob.Nvec

    Pb = likob.outlier_prob # a priori outlier probability for this sample
    P0 = likob.P0           # width of outlier range
    
    PA = 1.0 - Pb
    PB = Pb
    
    PtA = np.exp(-0.5*r**2/N) / np.sqrt(2*np.pi*N)
    PtB = 1.0/P0
    
    num = PtB * PB
    den = PtB * PB + PtA * PA
    
    return num/den, r/np.sqrt(N)

Here we compute the vector of outlier probabilities and their uncertainties, and save it to a .npy file

In [ ]:
pobsfile = psr + '-pobs.npy'

In [ ]:
%%time
if not os.path.isfile(pobsfile):
    nsamples = len(pars)
    nobs = len(likob.Nvec)

    # basic likelihood
    lo = likob

    outps = np.zeros((nsamples,nobs),'d')
    sigma = np.zeros((nsamples,nobs),'d')

    for i,p in enumerate(pars):
        outps[i,:], sigma[i,:] = poutlier(p,lo)

    out = np.zeros((nsamples,nobs,2),'d')
    out[:,:,0], out[:,:,1] = outps, sigma    
    np.save(pobsfile,out)
else:
    out = np.load(pobsfile)
    outps, sigma = out[:,:,0], out[:,:,1]

In [ ]:
avgps = np.mean(outps,axis=0)
medps = np.median(outps,axis=0)

Finally we can see where these outliers are (or if there even are any!). This will produce a typical plot of TOA residuals, but TOAs with a 10% median outlier probability will be highlighted. If there are no TOAs that reach this criterion, then any with >0.05% probability will be highlighted. Both of these thresholds can be changed at will.

In [ ]:
spd = 86400.0   # seconds per day
T0 = 53000.0        # reference MJD that is subtracted off all TOAs when processing through libstempo
residualplot = psr + '-residuals.pdf'

if not os.path.isfile(residualplot):
    outliers = medps > 0.1
    nout = np.sum(outliers)
    nbig = nout
    
    print("Big: {}".format(nbig))
    
    if nout == 0:
        outliers = medps > 5e-4
        nout = np.sum(outliers)
    
    print("Plotted: {}".format(nout))

    plt.figure(figsize=(15,6))

    psrobj = likob.psr

    # convert toas to mjds
    toas = psrobj.toas/spd + T0

    # red noise at the starting fit point
    _, _ = likob.full_loglikelihood_grad(endp)
    rednoise = psrobj.residuals - likob.detresiduals

    # plot tim-file residuals (I think)
    plt.errorbar(toas,psrobj.residuals,yerr=psrobj.toaerrs,fmt='.',alpha=0.3)

    # red noise
    # plt.plot(toas,rednoise,'r-')

    # possible outliers
    plt.errorbar(toas[outliers],psrobj.residuals[outliers],yerr=psrobj.toaerrs[outliers],fmt='rx')

    plt.savefig(residualplot)

If you want to know the exact indices of the highlighted TOAs (and their corresponding outlier probabilities), you can run the block below to print them out. This can (and probably should) be rewritten to just save this info to another file.

In [ ]:
for ii, elem in enumerate(outliers):
    if elem:
        print(f'Outlier TOA index: {ii}')
        print(f'Outlier probability: {medps[ii]}')